## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

C:\Users\Jason\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Jason\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Jason\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Jason\anaconda3\envs\PythonData\lib\site-packages

In [2]:
#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("../Resources/charity_data.csv").set_index('EIN')
df.head(2)

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
EIN,,,,,,,,,,,
10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1


In [3]:
df[['CLASSIFICATION','AFFILIATION']].duplicated().value_counts()

True     34189
False      110
dtype: int64

In [4]:
df[['CLASSIFICATION','USE_CASE']].duplicated().value_counts()

True     34197
False      102
dtype: int64

In [5]:
df[['CLASSIFICATION','ORGANIZATION']].duplicated().value_counts()

True     34173
False      126
dtype: int64

In [6]:
df[['CLASSIFICATION','STATUS']].duplicated().value_counts()

True     34225
False       74
dtype: int64

In [7]:
df[['CLASSIFICATION','INCOME_AMT']].duplicated().value_counts()

True     34081
False      218
dtype: int64

In [8]:
df[['CLASSIFICATION','SPECIAL_CONSIDERATIONS']].duplicated().value_counts()

True     34223
False       76
dtype: int64

In [9]:
drop_columns = ['NAME', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']


In [10]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df.drop(columns=drop_columns, inplace=True)

In [11]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE      17
CLASSIFICATION        71
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [12]:
# Look at APPLICATION_TYPE value counts for binning
vc = df['APPLICATION_TYPE'].value_counts()
vc

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [13]:
# Choose a cutoff value and create a list of application types to be replaced
app_repl = vc.loc[lambda x: x<= vc[5]].reset_index()['index']
app_repl

0      T8
1      T7
2     T10
3      T9
4     T13
5     T12
6      T2
7     T25
8     T14
9     T29
10    T15
11    T17
Name: index, dtype: object

In [14]:
# Replace in dataframe
for val in app_repl:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(val,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [15]:
# Look at CLASSIFICATION value counts for binning
vc = df['CLASSIFICATION'].value_counts()
vc[0:11]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [16]:
# You may find it helpful to look at CLASSIFICATION value counts >1
cls_repl = vc.loc[lambda x: x< vc[5]].reset_index()['index']
cls_repl

0     C1700
1     C4000
2     C5000
3     C1270
4     C2700
5     C2800
6     C7100
7     C1300
8     C1280
9     C1230
10    C1400
11    C7200
12    C2300
13    C1240
14    C8000
15    C7120
16    C1500
17    C1800
18    C6000
19    C1250
20    C8200
21    C1238
22    C1278
23    C1235
24    C1237
25    C7210
26    C1720
27    C2400
28    C4100
29    C1600
      ...  
35    C1234
36    C1256
37    C1246
38    C3200
39    C2170
40    C4200
41    C1900
42    C8210
43    C1248
44    C2150
45    C2561
46    C2600
47    C1245
48    C1580
49    C2500
50    C2570
51    C1570
52    C1236
53    C2190
54    C2380
55    C5200
56    C1728
57    C1370
58    C4120
59    C3700
60    C6100
61    C1283
62    C1820
63    C4500
64    C1732
Name: index, Length: 65, dtype: object

In [17]:
# Replace in dataframe
for val in cls_repl:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(val,"Other")

# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [18]:
df.head(2)

,APPLICATION_TYPE,CLASSIFICATION,ASK_AMT,IS_SUCCESSFUL
EIN,,,,
10520599,Other,C1000,5000,1
10531628,T3,C2000,108590,1


In [19]:
# Convert categorical data to numeric with `pd.get_dummies`
dums = pd.get_dummies(df)
print(dums.shape)
dums.head(2)

(34299, 15)


,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other
EIN,,,,,,,,,,,,,,,
10520599,5000,1,1,0,0,0,0,0,1,0,0,0,0,0,0
10531628,108590,1,0,0,1,0,0,0,0,0,1,0,0,0,0


In [20]:
# Split our preprocessed data into our features and target arrays
y = dums['IS_SUCCESSFUL']
X = dums.drop(columns = 'IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12345)

In [21]:
scale_cols = ['ASK_AMT']

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

ct = ColumnTransformer([
        ('ct', StandardScaler(), scale_cols)
     ], remainder='passthrough')

X_train_scaled = ct.fit_transform(X_train)
X_test_scaled = ct.fit_transform(X_test)

## Compile, Train and Evaluate the Model

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
layers = [
    {'units' : 100, 'activation': 'relu', 'input_dim': X_train_scaled.shape[1]},
    {'units' : 50, 'activation': 'relu'},
    {'units' : 50, 'activation': 'relu'},
    {'units' : 50, 'activation': 'relu'},
    {'units' : 1, 'activation': 'sigmoid'}
]

In [23]:

nn = tf.keras.models.Sequential()

for layer in layers:
    nn.add(tf.keras.layers.Dense(**layer))

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1500      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 11,701
Trainable params: 11,701
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [25]:
# Train the model
model = nn.fit(X_train_scaled, y_train, epochs = 200)

Epoch 1/200
25724/25724 [==============================] - 1s 38us/sample - loss: 0.6526 - acc: 0.6054
Epoch 2/200
25724/25724 [==============================] - 1s 33us/sample - loss: 0.6471 - acc: 0.6116
Epoch 3/200
25724/25724 [==============================] - 1s 30us/sample - loss: 0.6465 - acc: 0.6120
Epoch 4/200
25724/25724 [==============================] - 1s 30us/sample - loss: 0.6465 - acc: 0.6130
Epoch 5/200
25724/25724 [==============================] - 1s 29us/sample - loss: 0.6465 - acc: 0.6108
Epoch 6/200
25724/25724 [==============================] - 1s 30us/sample - loss: 0.6461 - acc: 0.6130
Epoch 7/200
25724/25724 [==============================] - 1s 33us/sample - loss: 0.6459 - acc: 0.6144
Epoch 8/200
25724/25724 [==============================] - 1s 29us/sample - loss: 0.6457 - acc: 0.6141
Epoch 9/200
25724/25724 [==============================] - 1s 37us/sample - loss: 0.6454 - acc: 0.6146
Epoch 10/200
25724/25724 [==============================] - 1s 32us/sampl

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.6543 - acc: 0.6132
Loss: 0.6543292408295345, Accuracy: 0.6131778359413147


In [27]:
# Export our model to HDF5 file
nn.save('minimal_hyperparameters_model.h5')